In [1]:
import requests

class Tistory:
    def __init__(self):
        self.defaultParam = {
            'access_token' : '',
            'output' : 'json'
        }
        from secrets import Secret
        self.secret = Secret()
    def getAuthCode(self):
        # this section will remake at frontend. this is poc code
        # change redirectUrl for realservice url later
        authCodeUrl = "https://www.tistory.com/oauth/authorize"
        params = {
            "client_id": self.secret.tistoryAppId,
            "redirect_uri": "http://ykarma1996.tistory.com/",
            "response_type": "code"
        }
        res = requests.get(authCodeUrl, params=params)
        
        # action this url at browser, and get AccessToken manually just now.
        print (res.url)
        
    def getAccessToken(self, authCode="getFromClient"):
        authCode = input("authCode : ") # delete this later
        
        getTokenUrl = "https://www.tistory.com/oauth/access_token"
        params = {
            "client_id": self.secret.tistoryAppId,
            "client_secret": self.secret.tistorySecretKey,
            "redirect_uri": "http://ykarma1996.tistory.com/",
            "code": authCode,
            "grant_type": "authorization_code"
        }
        res  = requests.get(getTokenUrl, params=params)
        self.defaultParam['access_token'] = res.text.split('=')[1]
    
    def getBlogList(self):
        params = self.defaultParam
        getBlogUrl = "https://www.tistory.com/apis/blog/info"
        res = requests.get(getBlogUrl, params=params)
        
        # Object of key: 'blogName' value: 'blogTitle'
        blogList = {}
        for blog in res.json()['tistory']['item']['blogs']:
            blogList[blog['name']] = blog['title']
        
        # Show to FrontEnd
        print (blogList)
        
    def getcatego(self, blogName):
        params = self.defaultParam
        params['blogName'] = blogName
        getcategoUrl = 'https://www.tistory.com/apis/category/list'
        res = requests.get(getcategoUrl, params=params)
        
        # Object of key : 'catego id' value: 'catego name'
        categoList = {}
        for catego in res.json()['tistory']['item']['categories']:
            categoList[catego['id']] = catego['name']
        
        print (categoList)
    
    def postContent(self, blogName, title, content, visibility, category, tag, acceptComment):
        params = self.defaultParam
        params['blogName'] = blogName
        params['title'] = title
        params['content'] = content
        params['visibility'] = visibility
        params['category'] = category
        params['tag'] = tag
        params['acceptComment'] = acceptComment
        
        postUrl = 'https://www.tistory.com/apis/post/write'
        res = requests.post(postUrl, data=params)
        
        print (res.json()['tistory']['url'])


In [4]:
from secrets import Secret
from makeHtml import MakeHtml

secret = Secret()
email = secret.email
apiKeyConfluence = secret.apiKeyConfluence
apiKeyImgbb = secret.apiKeyImgbb
userName = 'geunsam2'

make1 = MakeHtml()
make1.loginProcess(email, apiKeyConfluence, userName, apiKeyImgbb)
make1.getSpaceList()
lists = make1.getContentList("GEUNSAM2")

make1.getCententHtml('8880168')
make1.rebuildFormat()
make1.rebuildImgStore()
contents = str(make1.htmlSoup)

<h3 id="id-[confluencetotistory-1]API조사-단위목표">단위목표</h3><ul><li><p>confluence api 조사</p></li><li><p>페이지, 스페이스 목록 정보 조회 api 확인</p></li><li><p>특정 페이지 <code>html</code> or <code>markdown</code> export api 확인 </p></li></ul><p></p><h3 id="id-[confluencetotistory-1]API조사-confluenceapi조사">confluence api 조사</h3><ul><li><p>인증 구현</p></li></ul><ol><li><p>personal token 생성</p></li><li><p><code>&lt;로그인 이메일&gt;:&lt;토큰&gt;</code> 을 base64로 인코딩</p></li><li><p>Authorization 헤더에 Basic &lt;base64 인코딩 된 값&gt;을 추가하여 사용</p></li></ol><div class="code panel pdl conf-macro output-block" data-hasbody="true" data-macro-id="cd290408-af19-4bcb-8d92-9121104d976b" data-macro-name="code" style="border-width: 1px;"><div class="codeContent panelContent pdl">
<pre class="syntaxhighlighter-pre" data-syntaxhighlighter-params="brush: java; gutter: false; theme: Confluence" data-theme="Confluence">curl -H "Authorization: Basic &lt;yourToken&gt;" https://{confluenceBaseUrl}/rest/api/content</pre>
</div></div><ul><li><p>space 

In [7]:
def main():
    ti = Tistory()
    ti.getAuthCode()
    ti.getAccessToken()
    ti.getBlogList()
    
    #categoName = intput()
    blogName = 'ykarma1996'
    ti.getcatego(blogName)
    
    title = 'api 게시물 생성 테스트1'
    visibility = 0
    category = '512317' # freeCaht
    tag = 'testtag1,testtag2'
    acceptComment = 1
    
    ti.postContent(blogName, title, contents, visibility, category, tag, acceptComment)
    
if __name__ == "__main__":
    main()

https://www.tistory.com/auth/login/?popup=1&redirectUrl=https%3A%2F%2Fwww.tistory.com%2Foauth%2Fauthorize%3Fclient_id%3D1248d0d1dca9f950e085f8cff4bda0f4%26redirect_uri%3Dhttp%253A%252F%252Fykarma1996.tistory.com%252F%26response_type%3Dcode
authCode : 51c85710b4f1b37aea925626a4f5f450f16c3ec5cae5270fd03e1c1ee57bb31d846e9807
{'ykarma1996': '모두의 근삼이', 'moduboan': '모두의 보안'}
{'1107725': 'Marathon_Mesos', '982641': '리눅스', '1011834': '모두의 보안', '1039593': '모두의 보안 1기', '1184515': '개발 이것저것', '1107726': 'Kubernetes', '1039594': '모두의 보안 2기', '1107727': 'Jenkins', '1089407': 'DevOps', '1039595': '모두의 보안 3기', '1107728': 'Docker', '1107730': '미분류', '1107729': 'Ansible', '1011837': 'SYSTEM', '1107731': 'Database', '1011835': 'WEB', '512220': 'IT정보', '826015': '네트워크', '512216': '블로그', '512317': 'Free Chat', '512221': 'Diary'}
https://ykarma1996.tistory.com/114
